In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from urllib.request import urlopen
from bs4 import BeautifulSoup
import time

## Using Beautiful Soup

In [6]:
url = "https://www.oddsportal.com/rugby-union/world/super-rugby/"
html = urlopen(url, timeout = 5)

In [7]:
soup = BeautifulSoup(html, 'lxml')

In [8]:
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xml:lang="cs" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="Thu, 01 Jan 1970 00:59:59 GMT" http-equiv="expires"/>
  <meta content="no-cache" http-equiv="pragma"/>
  <meta content="no-cache" http-equiv="cache-control"/>
  <meta content="sport" property="og:type"/>
  <meta content="https://www.oddsportal.com/oddsportal.png" property="og:image"/>
  <meta content="Oddsportal.com" property="og:site_name"/>
  <link href="https://www.oddsportal.com/oddsportal.png" rel="image_src"/>
  <!-- APPLE TOUCH ICONS -->
  <link alt="Iphone icon" href="/res/img/apple-touch-icon.png" rel="apple-touch-icon"/>
  <link alt="Ipad icon" href="/res/img/apple-touch-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/>
  <link alt="Iphone, Ipad Hi icon" href="/res/img/apple-touch-icon-114x114.png" r

In [61]:
soup.find_all('tr',{'xeid':True})

[<tr class="odd deactivate" xeid="WrLmpx8N"><td class="table-time datet t1550817300-1-1-0-0 "></td><td class="name table-participant"><a href="/rugby-union/world/super-rugby/highlanders-queensland-reds-WrLmpx8N/">Highlanders - Reds</a></td><td class="center bold table-odds table-score">36:31</td><td class="odds-nowrp"><span>-</span></td><td class="odds-nowrp"><span>-</span></td><td class="odds-nowrp"><span>-</span></td><td class="center info-value"></td></tr>,
 <tr class="odd" xeid="E7AhqdNT"><td class="table-time datet t1550895300-1-1-0-0 "></td><td class="name table-participant" colspan="2"><a href="/rugby-union/world/super-rugby/sunwolves-waratahs-E7AhqdNT/">Sunwolves - Waratahs</a></td><td class="odds-nowrp"><span>-</span></td><td class="odds-nowrp"><span>-</span></td><td class="odds-nowrp"><span>-</span></td><td class="center info-value"></td></tr>,
 <tr xeid="jT1KvGVp"><td class="table-time datet t1550903700-1-1-0-0 "></td><td class="name table-participant" colspan="2"><a href="/

In [62]:
soup.find_all('tr',attrs={'xeid':True,
                         'class':'table-participant'})

[]

In [ ]:
# list of rows
list_rows = []

In [ ]:
# loop through table rows, remove html formatting, append to list
for row in all_rows:
  row_td = row.find_all('td', {'xparam': 'odds_text'})
  
  # r_cells = str(row_td)
  # leantext = BeautifulSoup(str_cells, "lxml").get_text()
  
  list_rows.append(cleantext)

In [35]:
list_rows

['[\n\n, \n\nGet VIP status!\n\n\n\t\t\t\t\t\t\t\t\t\tRegister via OddsPortal.com, verify your account and enjoy instant VIP account benefits!\n\t\t\t\t\t\t\t\t\t]',
 '[\n\n, \n\nGet VIP status!\n\n\n\t\t\t\t\t\t\t\t\t\tRegister via OddsPortal.com, verify your account and enjoy instant VIP account benefits!\n\t\t\t\t\t\t\t\t\t]',
 '[\n\n, \n\nGet VIP status!\n\n\n\t\t\t\t\t\t\t\t\t\tRegister via OddsPortal.com, verify your account and enjoy instant VIP account benefits!\n\t\t\t\t\t\t\t\t\t]',
 '[\n\n, \n\nGet VIP status!\n\n\n\t\t\t\t\t\t\t\t\t\tRegister via OddsPortal.com, verify your account and enjoy instant VIP account benefits!\n\t\t\t\t\t\t\t\t\t]',
 '[\n\n, \n\nGet VIP status!\n\n\n\t\t\t\t\t\t\t\t\t\tRegister via OddsPortal.com, verify your account and enjoy instant VIP account benefits!\n\t\t\t\t\t\t\t\t\t]',
 '[\n\n, \n\nGet VIP status!\n\n\n\t\t\t\t\t\t\t\t\t\tRegister via OddsPortal.com, verify your account and enjoy instant VIP account benefits!\n\t\t\t\t\t\t\t\t\t]',
 '[\

## Using Scrapy

In [9]:
import scrapy

ModuleNotFoundError: No module named 'scrapy'

In [ ]:
# create scrapy spider object
class BrickSetSpider(scrapy.Spider):
    name = "oddsportal_spider"
    start_urls = ['https://www.oddsportal.com/rugby-union/world/super-rugby/']

In [ ]:
# run spider
